# 1. SFT
## 던파에 대한 내용을 아는 모델을 만들자!

In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

In [ ]:
import os
import torch
import random
import argparse
import numpy as np

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

random_seed = 42
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)


In [ ]:
# define argment
parser = argparse.ArgumentParser()
parser.add_argument('--output_dir', type=str, default='./output_1_SFT')
parser.add_argument('--data_path', type=str, default='/content/drive/MyDrive/ds_study/던파_story/data/train_data/dnf_story_forTrain_tokenFor1024.jsonl')
parser.add_argument('--model', type=str, default='gpt2', choices=['gpt2', 'bloom', 'opt'])
parser.add_argument('--pretrain', type=str, default=None)
parser.add_argument('--max_epochs', type=int, default=10)
parser.add_argument('--batch_size', type=int, default=8)

args = parser.parse_args(args=[])

# for test
args.max_epochs = 3
args.pretrain = 'EleutherAI/polyglot-ko-1.3b'  # pretrained 모델 가져오기
args.verbose = True

print(args)
if not os.path.exists(args.output_dir):
    os.makedirs(args.output_dir)

In [ ]:
# 모델, 토크나이저
tokenizer = AutoTokenizer.from_pretrained(args.pretrain)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(args.pretrain, quantization_config=bnb_config, device_map={"":0})

In [ ]:
from datasets import load_dataset

# load SFT train data
data= load_dataset("json", data_files=args.data_path)

# fitting prefix form
data = data.map(
    lambda x: {'text': f"### 질문: {x['prompt']}\n\n### 답변: {x['completion']}<|endoftext|>" }
)

# encoding text
data = data.map(lambda samples: tokenizer(samples["text"]), batched=True)


In [ ]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)


In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)


model = get_peft_model(model, config)
print_trainable_parameters(model)

In [ ]:
import transformers

trainer = transformers.Trainer(
    model=model,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=args.batch_size,
        gradient_accumulation_steps=2,
        learning_rate=1e-4,
        fp16=True,
        logging_steps = 100,
        save_steps = 100,
        output_dir=args.out_dir,
        optim="paged_adamw_4bit",
        save_total_limit = 3,
        seed = random_seed
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

trainer.train()

# load checkpoint, continue train
# trainer.train(resume_from_checkpoint = '/content/drive/MyDrive/ds_study/던파_story/output_1_SFT/checkpoint-2800')

In [ ]:
# Model Save
trainer.save_model(args.out_dir)

### Generate Text finetuning model

In [ ]:
# if restart session have to run

# import torch
# from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# from peft import PeftModel, PeftConfig


# peft_model_id = args.out_dir
# config = PeftConfig.from_pretrained(peft_model_id)
# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.bfloat16
# )
# model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, quantization_config=bnb_config, device_map={"":0})
# model = PeftModel.from_pretrained(model, peft_model_id)
# tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# model.eval()

In [ ]:
import time

def gen(x):
    start_time = time.time()
    gened = model.generate(
        **tokenizer(
            f"### 질문: {x}\n\n### 답변:",
            return_tensors='pt',
            return_token_type_ids=False
        ).to(0),
        num_beams=3,
        no_repeat_ngram_size=4,
        top_p = 0.9,
        max_new_tokens=256,
        early_stopping=True,
        do_sample=True,
        eos_token_id=2,
    )
    print(tokenizer.decode(gened[0]))
    print(time.time() - start_time)

In [ ]:
# gen('건강하게 살기 위한 세 가지 방법은?')
list_prompt = ['일곱 사도들에 대해서 알려줘',
               '아라드 대륙에 대해서 설명해줘.',
               '바칼이 천계를 공격한 이유는?.',
               '게이볼그 프로젝트에 대해서 요약해주세요'
               ]

for input in list_prompt:
  gen(input)
  print()

# 2. RM

## 좋은 대답을 평가하는 모델을 만들자!


## 설치 모듈

In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets
!pip install jsonlines
!pip install trl

## 좋은 대답을 할 수 있게 유도하는 보상 모델을 만들자!


In [ ]:
from datasets import load_dataset
from peft import LoraConfig, TaskType
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import BitsAndBytesConfig, TrainingArguments
from trl import RewardTrainer, RewardConfig

import os
import json
import torch
import random
import argparse
import numpy as np


random_seed = 42
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)


In [ ]:
# define argment
parser = argparse.ArgumentParser()
parser.add_argument('--output_dir', type=str, default='/content/drive/MyDrive/ds_study/던파/output_2_RM')
parser.add_argument('--data_path_RM', type=str, default='/content/drive/MyDrive/ds_study/던파_story/data/train_data/kochatgpt_2_RM.jsonl')
parser.add_argument('--df_data_path_RM', type=str, default='/content/drive/MyDrive/ds_study/던파_story/data/train_data/dnf_data_RM.json')
parser.add_argument('--strategy',
                    choices=['naive', 'ddp', 'colossalai_gemini', 'colossalai_zero2'],
                    default='naive')
parser.add_argument('--model', type=str, default=None)
parser.add_argument('--pretrain', type=str, default=None)
parser.add_argument('--dataset', type=str, default='Dahoas/rm-static')
parser.add_argument('--save_path', type=str, default='rm_ckpt.pth')
parser.add_argument('--max_epochs', type=int, default=10)
parser.add_argument('--batch_size', type=int, default=4)
parser.add_argument('--lora_rank', type=int, default=0, help="low-rank adaptation matrices rank")
parser.add_argument('--max_len', type=int, default=512)  # wygo 추가

args = parser.parse_args(args=[])

# for test
args.max_epochs = 3
args.pretrain = "EleutherAI/polyglot-ko-1.3b" #huggingFace
# args.pretrain = 'skt/kogpt2-base-v2'

if not os.path.exists(args.output_dir):
    os.makedirs(args.output_dir)

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForSequenceClassification.from_pretrained(args.pretrain, quantization_config=bnb_config, device_map={"":0}, num_labels=1)

tokenizer = AutoTokenizer.from_pretrained(args.pretrain)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
# make ranking data to chosen, rejetced data
rm_data_combine = []
with open(args.df_data_path_RM, "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)

    print('## df_data check ##')
    print((list_data_dict[0]))

rm_data_combine += list_data_dict

with open(args.data_path_RM, "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)

    print('## data check ##')
    print((list_data_dict[0]))

rm_data_combine += list_data_dict

## df_data check ##
{'prompt': '바다와 전쟁의 흔적, 그리고 마계의 사도들에 대한 이야기를 요약 해주세요.', 'completion_0': '소녀는 처음으로 넓은 바다를 보고 감탄하며 그 아름다움에 매료되었습니다. 그러나 그녀가 바라본 바다 주변의 섬은 과거 전쟁으로 황폐해진 곳이었습니다. 이곳에서 바다와 다른 풍경을 이루는 활화산이 있는 섬에 사도 안톤이 존재했다는 사실에 놀라움을 감추지 못했습니다. 소녀는 마계와 천계를 이어주는 통신 장치를 통해 연결된 리아 리히터와 통신을 하며, 자신이 마계에서 사라진 사도들을 찾는 데에 중요한 역할을 할 것이라는 기대감을 주고받습니다. 그러나 갑자기 차원의 틈이 닫혀 통신은 끊기고, 이제 그녀는 천계와 마계의 경계에서 불확실한 미래를 향해 자신의 역할을 다하기 위해 각오를 다집니다.', 'completion_1': '바다를 처음 본 소녀는 과거 전쟁터였던 섬의 아름다움에 놀랍니다. 그리고 통신기를 통해 리아와 사도 안톤의 존재 및 상황을 나누며 정보를 주고받지만, 차원의 틈이 닫혀 다시 연락을 기다려야 하는 상황에 처하게 됩니다.', 'completion_2': '바다에 감탄한 소녀가 마계의 사도들과 관련한 중요한 통신을 하다 차원의 틈이 닫혀 소통이 끊깁니다.', 'ranking': [0, 2, 1]}
## data check ##
{'prompt': '번디는 자신이 탐정잡지, 범죄소설 그리고 성범죄 관련 실제 범죄 다큐멘터리들을 탐독했다고 누구에게 말했나?', 'completion_0': 'Allow me to answer your question. I know that you are curious about me.', 'completion_1': '번디는 다양한 인터뷰자들과 뉴스홍보 담당자들과의 면담 때 밝혔다.', 'completion_2': '라이언에게 말했다.', 'ranking': [2, 1, 0]}


In [ ]:
total_data_ranking2chosen = []
for tmp in rm_data_combine:
    one_data_ranking2chosen = []

    # data 1) 0 VS 1
    data = {}
    data['prompt'] = tmp['prompt']
    if tmp['ranking'][0] < tmp['ranking'][1]:
        data['chosen'] = tmp['completion_0']
        data['rejected'] = tmp['completion_1']
    else:
        data['chosen'] = tmp['completion_1']
        data['rejected'] = tmp['completion_0']
    one_data_ranking2chosen.append(data)


    # data 2) 0 VS 2
    data = {}
    data['prompt'] = tmp['prompt']
    if tmp['ranking'][0] < tmp['ranking'][2]:
        data['chosen'] = tmp['completion_0']
        data['rejected'] = tmp['completion_2']
    else:
        data['chosen'] = tmp['completion_2']
        data['rejected'] = tmp['completion_0']
    one_data_ranking2chosen.append(data)

    # data 1) 1 VS 2
    data = {}
    data['prompt'] = tmp['prompt']
    if tmp['ranking'][1] < tmp['ranking'][2]:
        data['chosen'] = tmp['completion_1']
        data['rejected'] = tmp['completion_2']
    else:
        data['chosen'] = tmp['completion_2']
        data['rejected'] = tmp['completion_1']
    one_data_ranking2chosen.append(data)



    total_data_ranking2chosen.extend(one_data_ranking2chosen)

print('before data num: %d'%(len(rm_data_combine)))
print('after  data num: %d'%(len(total_data_ranking2chosen)))
print('data example: \n%s'%total_data_ranking2chosen[45])

before data num: 12988
after  data num: 38964
data example: 
{'prompt': "'정복자 카시야스'는 누구인가요?", 'chosen': "'정복자 카시야스'는 호전적인 귀면족의 일원으로 마계의 에컨에서 태어났으며, 강한 기운을 찾아 전투를 치르는 존재입니다. 이 종족은 평범한 인간보다 약 1.5배 큰 체구를 가지고 있으며, 두 자루의 검을 사용하여 싸웁니다. 카시야스는 사도 중에서도 최고의 전투 실력을 자랑하지만, 제 1사도인 숙명의 카인에게는 패하여 더 강해지기 위해 마계 소환사와 계약을 맺고 아라드 대륙으로 건너옵니다. 새로운 세계에서 새로운 적들과의 대결을 통해 자신을 강화하며, 항상 새로운 도전을 기대하는 낙천주의자입니다. 성우 안장혁이 이 캐릭터의 목소리를 담당했습니다.", 'rejected': "'정복자 카시야스'는 마계에 살던 귀면족으로 강함을 추구하여 아라드로 온 전사입니다. 최강이라는 명성에 걸맞게 다양한 전투 경험을 쌓으나 1사도에게 패배하고, 이를 계기로 끝없이 강해지고자 합니다. 그는 또한 끊임없는 새로운 도전을 좋아하는 낙천적 기질을 가지고 있습니다."}


In [ ]:
def formatting_func(examples):
    kwargs = {"padding": "max_length", "truncation": True, "max_length": 512, "return_tensors": "pt"}
    chosen = examples["prompt"] + "\n" + examples["chosen"]
    rejected = examples["prompt"] + "\n" + examples["rejected"]

    tokens_chosen = tokenizer.encode_plus(chosen, **kwargs)
    tokens_rejected = tokenizer.encode_plus(rejected, **kwargs)

    return {
        "input_ids_chosen": tokens_chosen["input_ids"][0], "attention_mask_chosen": tokens_chosen["attention_mask"][0],
        "input_ids_rejected": tokens_rejected["input_ids"][0], "attention_mask_rejected": tokens_rejected["attention_mask"][0]
    }
from datasets import Dataset
train_data = Dataset.from_list(total_data_ranking2chosen[:-3000])
eval_data = Dataset.from_list(total_data_ranking2chosen[-3000:])

train_data.to_pandas()
eval_data.to_pandas()

train_dataset = train_data.map(formatting_func)
eval_dataset = eval_data.map(formatting_func)

Map:   0%|          | 0/35964 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [ ]:
from peft import LoraConfig, get_peft_model

peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,
    bias = 'none'
)

model = get_peft_model(model, peft_config)


In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )
print_trainable_parameters(model)

trainable params: 1574912 || all params: 667803648 || trainable%: 0.2358345907089145


In [ ]:
for param in model.parameters():
    if param.requires_grad:
        param.data = param.data.to(torch.float32)

In [ ]:
import gc
torch.cuda.empty_cache()
gc.collect()

In [ ]:
training_args = TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=8,
        learning_rate=1e-4,
        fp16=True,
        logging_steps = 100,
        save_steps = 100,
        output_dir=args.output_dir,
        optim="paged_adamw_8bit",
        save_total_limit = 3,
        seed = random_seed
    )


trainer = RewardTrainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
model.config.pad_token_id = model.config.eos_token_id

trainer.train()

# Load checkpoint, contiue train
# trainer.train(resume_from_checkpoint = '/content/drive/MyDrive/ds_study/던파_story/output_2_RM/checkpoint-3000')

In [ ]:
# model save
model.save_pretrained(args.output_dir)

### Check Reward Score, RM


In [ ]:
import torch
from transformers import AutoTokenizer, BitsAndBytesConfig, AutoModelForSequenceClassification

# inference Score
def inference_RM(model, rm_model, input_text):

    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(
        torch.cuda.current_device())

    output = model(input_ids)

    output_reward = output.cpu().detach().numpy()[0]

    print('input: %s\nreward score: %.1f'%(input_text, output_reward))
    print('df_reward score: %.1f'%(df_output_reward))

    return output_reward


In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
rm_adapter = args.output_dir
origin_model = "EleutherAI/polyglot-ko-1.3b" #huggingFace

model = AutoModelForSequenceClassification.from_pretrained(
    rm_adapter,
    quantization_config=bnb_config,
    num_labels=1)

tokenizer = AutoTokenizer.from_pretrained(origin_model)


input_texts = ['한국에서 사는건 어떤가요?\n한국에서 사는 것은 행복한 일입니다.',
               '왜 점수가 안바뀌는거죠?\n제가 대답할 수 있는 질문이 아닙니다.',
               '핀드에 대해서 알려줘\n융합형 핀드는 불 속성과 물 속성의 속성을 가지고 있습니다. 이 두 속성의 융합으로 인해 융합형 핀드는 정신적으로 불안한 상태에 놓여있지만, 불 속성의 성격과 물 속성의 성격을 모두 가지는 특징을 가지고 있습니다.',
               ]

for input_text in input_texts:
  print('-'*70)
  inference_RM(model, rm_model, input_text=input_text)
  torch.cuda.empty_cache()
  print()

REWARD MODEL
input: 한국에서 사는 것은 행복한 일입니다.
reward score: 1.2
df_reward score: 0.1

REWARD MODEL
input: 왜 점수가 안바뀌는거죠?
reward score: 1.2
df_reward score: 0.1

REWARD MODEL
input: 융합형 핀드는 불 속성과 물 속성의 속성을 가지고 있습니다. 이 두 속성의 융합으로 인해 융합형 핀드는 정신적으로 불안한 상태에 놓여있지만, 불 속성의 성격과 물 속성의 성격을 모두 가지는 특징을 가지고 있습니다.
reward score: 1.1
df_reward score: 0.1

REWARD MODEL
input: '플레인 : 인퍼널'에서 이단심판관들은 위장자가 된 세계에서 자신들을 저주받은 불꽃이라 칭하며 자신들만의 길을 걷기 시작했습니다. 이들은 혼돈의 신에게 복종하지 않았지만, 여전히 신을 향한 믿음만큼은 갖고 있었습니다. 그러나 위장자의 기운으로 인해 불꽃이 검보라빛으로 변하면서 더욱 강력해졌습니다. 마침내 모든 위장자를 소멸시키고 아라드에 평화를 가져다줬습니다.
reward score: 1.2
df_reward score: 0.1



# 3.PPO
## PPO를 활용해 더 좋은 대답을 하도록 학습시키자

In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets
!pip install trl

In [1]:
# imports
import torch
from transformers import AutoTokenizer, BitsAndBytesConfig, AutoModelForSequenceClassification
from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead, create_reference_model
from trl.core import respond_to_batch
from peft import LoraConfig
from datasets import load_dataset, Dataset

import os
import json
import numpy as np
import random
from tqdm import tqdm


random_seed = 42
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)


In [2]:
PROMPT_TEXT = "### 질문: {txt}\n\n### 답변: "

# make dataset PPO
def build_dataset(dataset_path, tokenizer):

  ds = load_dataset('json', data_files = dataset_path)
  tmp = []
  for p in ds['train']:
    if p['prompt'] != '':
      tmp.append(p)
  ds = Dataset.from_list(tmp)

  def tokenize(sample):
    prompt = PROMPT_TEXT.format(txt = sample['prompt'])
    sample['input_ids'] = tokenizer.encode(prompt)
    sample['query'] = prompt
    return sample

  ds = ds.map(tokenize, batched=False)
  ds.set_format(type='torch')
  return ds

# make input stlye SFT
def make_input(txt):
  prompt_text = PROMPT_TEXT.format(txt = txt)
  encode = tokenizer(prompt_text, return_token_type_ids=False, return_tensors='pt').to('cuda')
  return encode

# 보상모델 체크
def inference_RM(model, input_text):
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to('cuda')

    output = model(input_ids)

    output_reward = (output.logits.cpu().detach().numpy())

    return torch.Tensor(output_reward[0])


In [ ]:
model_name = "/content/drive/MyDrive/ds_study/던파_story/output_1_SFT/model_QnA_FIN2"
rm_adapter_id = "/content/drive/MyDrive/ds_study/던파_story/output_2_RM/RM_FIN"
origin_model = "EleutherAI/polyglot-ko-1.3b" #huggingFace

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# PPO adapter
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

# Load SFT Model
model = AutoModelForCausalLMWithValueHead.from_pretrained(
    model_name,
    peft_config=lora_config,
    quantization_config=bnb_config,
)

# QLoRA를 통한 로드 모델은 deepcopy가 구현되지 않음 직접 하나 더 불러와야함
# model_ref = create_reference_model(model) # <--- TRL패키지에서도 copy.deepcopy()를 사용해 모델을 복사하고 있기 때문에 작동하지 않음. 이유는 QLoRA를 통한 로드를 고려하지 않았기 때문 수정진행중이라는 github issue확인.
model_ref = AutoModelForCausalLMWithValueHead.from_pretrained(
    model_name,
    peft_config=lora_config,
    quantization_config=bnb_config,
)

# Load RM
rm_model = AutoModelForSequenceClassification.from_pretrained(
    rm_adapter_id,
    quantization_config=bnb_config,
    num_labels=1)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(origin_model)

# model to cuda
# model.to('cuda')
# model_ref.cuda()

In [ ]:
# load PPO dataset
rm_jsonl_path= '/content/drive/MyDrive/ds_study/던파_story/data/train_data/dnf_data_RM.json'
dataset = build_dataset(rm_jsonl_path, tokenizer)

In [5]:
def collator(data):
    return dict((key, [d[key] for d in data]) for key in data[0])

# initialize trainer
ppo_config = PPOConfig(
    batch_size=2,
)
ppo_trainer = PPOTrainer(ppo_config, model, model, tokenizer, dataset=dataset, data_collator = collator)

In [6]:
step = 0

for epoch, batch in tqdm(enumerate(ppo_trainer.dataloader)):
  query = batch['query']
  query_tensors = batch["input_ids"]
  query_tensors = [tensor for tensor in query_tensors]
  query_encode = make_input(query)

  # get model response
  response_tensors = []
  response_tensors.append(gen(query_encode, model))
  batch["response"] = [tokenizer.decode(r.squeeze()) for r in response_tensors]

  # get RM score
  rm_inputs = [q + '\n' + r for q, r in zip(batch["query"], batch["response"])] # query_txt + '\n' + tokenizer.decode(response_tensor)
  rewards = []
  for rm_input in rm_inputs:
    rewards.append(inference_RM(rm_model, rm_input))

  # PPO
  stats = ppo_trainer.step(query_tensors, response_tensors, rewards)
  ppo_trainer.log_stats(stats, batch, rewards)
  step+=1

  # make checkpoint
  if step % 50 == 0:
    checkponit = '/content/drive/MyDrive/ds_study/던파_story/output_3_PPO/checkpoint-' + str(step)
    if not os.path.exists(checkponit):
      os.makedirs(checkponit)

    model.save_pretrained(checkponit)




0it [00:00, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:434: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
131it [21:33,  9.89s/it]/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_trainer.py:1279: UserWarning: KL divergence is starting to become negative: -4.06 - this might be a precursor for failed training. sometimes this happens because the generation kwargs are not correctly set. Please make sure that the generation kwargs are set correctly, or review your training hyperparameters.
  warnings.warn(
643it [1:44:43,  9.72s/it]/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_trainer.py:1279: UserWarning: KL divergence is starting to become negative: -1.27 - this might be a precursor for failed training. sometimes this happens because the generation kwargs are 

In [ ]:
generation_kwargs = {
    "min_length": -1,
    'max_new_tokens' : 128,
    "top_p": 0.95,
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id,
    'eos_token_id' : 2,
    'early_stopping' : True,
    'num_beams' : 1,
    'no_repeat_ngram_size' : 4
}
epoch = 1
TARGET_EPOCH = 1
while True:

  for train_step, batch in tqdm(enumerate(ppo_trainer.dataloader)):
    queries = batch['query']
    query_tensors = batch["input_ids"]
    query_tensors = [tensor for tensor in query_tensors]

    # get model response
    response_tensors = []
    for query in (query_tensors):
      query_len = len(query)
      response = ppo_trainer.generate(query, **generation_kwargs)
      end_point = (torch.flip(response.squeeze(), [0]) == 17).nonzero(as_tuple=False)[0] # tokenizer '.' token_id = 17
      response_tensors.append(response.squeeze()[:-end_point][query_len:])
    batch["response"] = [tokenizer.decode(r.squeeze()) for r in response_tensors]

    # get RM score
    rm_inputs = [q + '\n' + r for q, r in zip(batch["query"], batch["response"])] # query_txt + '\n' + tokenizer.decode(response_tensor)
    rewards = []
    for rm_input in rm_inputs:
      rewards.append(inference_RM(rm_model, rm_input))

    # break
    # PPO
    stats = ppo_trainer.step(query_tensors, response_tensors, rewards)
    ppo_trainer.log_stats(stats, batch, rewards)

    # make checkpoint
    if (train_step+1) % 50 == 0:
      checkponit = '/content/drive/MyDrive/ds_study/던파_story/output_3_PPO/checkpoint-' + str(train_step+1)
      if not os.path.exists(checkponit):
        os.makedirs(checkponit)

      model.save_pretrained(checkponit)

  # check Epoch
  if epoch == TARGET_EPOCH:

    print('\nEpoch 1 FIN\n')
    break
  epoch+=1



In [7]:
out_dir = "/content/drive/MyDrive/ds_study/던파_story/output_3_PPO/PPO_FIN"
if not os.path.exists(out_dir):
      os.makedirs(out_dir)
model.save_pretrained(out_dir)

### Compare PPO_SFT, ref_SFT

In [9]:
#### get a batch from the dataset
bs = 16
game_data = dict()
dataset.set_format("pandas")
df_batch = dataset[:].sample(bs)
game_data["query"] = df_batch["query"].tolist()
query_tensors = df_batch["input_ids"].tolist()


response_tensors_ref, response_tensors = [], []

#### get response from PPO_SFT, ref_SFT
for i in tqdm(range(bs)):
    output = gen(make_input(game_data['query'][i]), model_ref)
    response_tensors_ref.append(output)

    output = gen(make_input(game_data['query'][i]), model)
    response_tensors.append(output)


#### decode responses
game_data["response (before)"] = [tokenizer.decode(response_tensors_ref[i]) for i in range(bs)]
game_data["response (after)"] = [tokenizer.decode(response_tensors[i]) for i in range(bs)]

#### sentiment analysis of query/response pairs before/after
tmp = []
texts = [q + r for q, r in zip(game_data["query"], game_data["response (before)"])]
for txt in texts:
  tmp.append(inference_RM(rm_model, txt))
game_data["rewards (before)"] = tmp

tmp =[]
texts = [q + r for q, r in zip(game_data["query"], game_data["response (after)"])]
for txt in texts:
  tmp.append(inference_RM(rm_model, txt))
game_data["rewards (after)"] = tmp

import pandas as pd

# store results in a dataframe
df_results = pd.DataFrame(game_data)
df_results

  0%|          | 0/16 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:434: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
100%|██████████| 16/16 [03:46<00:00, 14.17s/it]


,query,response (before),response (after),rewards (before),rewards (after)
0,타우킹 샤우타는 누구인가요?,"타우킹 사도들은 샤우타를 타우로 부르고, 그들은 그를 타우킹이라 칭합니다. 이들은...",타우킹 타우는 타우타의 조상입니다. 원래는 불을 잘 다루었지만 그 능력이 점차 쇠...,[tensor(0.6245)],[tensor(3.5957)]
1,'오필리아 베이그란스'는 어떤 종교 집단의 신도인가요?,"'오필리어 베이그란스는 루크를 따르는 자들 중 하나로, 현재는 교단에서 운영하는 ...",'오필리오 베이그란스'()는 황도군에 합류한 후에도 종교 단체에 남아 있었습니다....,[tensor(3.3848)],[tensor(2.3184)]
2,에너지 생산실은 어떤 기능을 가지고 있나요?,"에너지 생산실은 에너지 저장과 전송의 기능을 가지며, 각 에너지는 다양한 형태로 ...",에너지 생산실은 검은 대지를 관리하는 차원의 균열의 에너지를 다루는 곳입니다. 이...,[tensor(-0.5986)],[tensor(2.6523)]
3,'람바녹'은 어떤 사람이었나요?,"람바녹은 그란플로리스의 주인으로, 검은 성전에 참가하여 모든 무기들을 파괴하고 많...",람바녹은 천계의 다른 모험가들과 달리 자신의 경험을 살려 무기를 다루는 법을 알려...,[tensor(5.9219)],[tensor(4.1758)]
4,'머크우드'는 어떤 곳인가요?,"'머크 우드'는 마계를 상징하는 지역 중 하나로, 마계의 중심지인 마계 도시입니다...","'머크 우드'는 노스피스의 북쪽으로, 황량한 사막과 끝없이 펼쳐진 바다가 끝없이 ...",[tensor(3.6680)],[tensor(1.8340)]
5,'플레인 : 코스모핀드'는 무엇인가요?,'플레인: 코스모필스'는 검은 마물의 후예이자 검은 마검사인 코스모 핀드를 지칭합...,'플레인 코스모필스'는 플레인에 거주하는 코스모핀이라는 존재입니다. 그는 다른 코...,[tensor(4.4531)],[tensor(4.0039)]
6,헤블론 바실리움을 만든 주인은 누구인가요?,헤블론의 헤블론을 만든 주인입니다. 헤블롱은 자신을 위해서 죽은 주인을 기리기 ...,헤블론의 주인은 헤블론 안에서만 살아가는 헤블론을 발견하고 헤블론에 머물고 싶어하...,[tensor(2.8164)],[tensor(3.8770)]
7,'네빌로 유르겐 행적'에 대해서 어떤 내용을 찾을 수 있나요?,본문에서는 네빌로 유를 겐트에 남은 마지막 귀족으로 언급하면서 그의 행적을 다루고...,네빌로 유를 찾기 위해서는 네빌로라는 인물의 행적을 알아야 합니다. 네빌로는 유르...,[tensor(1.3340)],[tensor(0.4771)]
8,'칠흑의 송곳니'는 어떤 작품인가요?,"'칠흑'은 선계 3대 아트의 하나인 ""칠흑의 분노""를 주제로 한 작품입니다. 이 ...",'칠흑'은 천계의 하늘을 방황하는 악귀들을 표현한 작품입니다. 이 작품은 선한 마...,[tensor(1.2354)],[tensor(5.0312)]
9,어떤 내용에 대한 요약을 진행하였습니까?,"선민의식이라는 소재를 중심으로 하여, 이는 그의 작품 전반에 흐르는 주요 특징 ...","최근에 모험가가 경험한 이야기로, 이야기 속에서 자신이 누구와 같이 모험을 떠나는...",[tensor(1.5322)],[tensor(6.2031)]


In [14]:
for idx, row in df_results[:5].iterrows():
  print("Query : ", row["query"])
  print("Response (before) : ", row["response (before)"])
  print("Response (after) : ", row["response (after)"])
  print("Rewards (before)/(after) : ", row["rewards (before)"],'/',row["rewards (after)"])
  print()

Query :  타우킹 샤우타는 누구인가요?
Response (before) :   타우킹 사도들은 샤우타를 타우로 부르고, 그들은 그를 타우킹이라 칭합니다. 이들은 샤우드의 우두머리로서 역할을 수행합니다. 또한, 타우킹은 타우라는 이름의 사도에 의해 소환되는 존재로서 그의 권능을 통해 자신의 권능으로 다른 존재를 제어하고 지배할 수 있습니다. 그들은 샤우트의 힘에 의해 타우라는 힘을 부여받고 그의 능력을 사용하여 타우킹으로 불리며, 타우의 권능에 의해 다른 개체들을 지배할 수는 없지만
Response (after) :   타우킹 타우는 타우타의 조상입니다. 원래는 불을 잘 다루었지만 그 능력이 점차 쇠퇴해지면서 현재는 타우킹이 되었습니다. 이들은 주로 숲에서 활동하며 타우타와 같이 나무를 자릅니다. 타우타는 나무를 잘 다루며 숲을 지키기 위해 노력하고 있습니다.  타우타가 자신의 모습을 본떠 만들어졌다고 말하는 이가 있지만 타우타 자체는 타우타나 타우타를 본떠 만들지 않았습니다. 하지만 타우타에 대한 전설이나 이야기들이 전해
Rewards (before)/(after) :  tensor([0.6245]) / tensor([3.5957])

Query :  '오필리아 베이그란스'는 어떤 종교 집단의 신도인가요?
Response (before) :   '오필리어 베이그란스는 루크를 따르는 자들 중 하나로, 현재는 교단에서 운영하는 특수 조직의 일원입니다. 그녀는 루크의 특별한 능력을 받았으며, 이 능력으로 자신들을 이끌어 루크의 제국을 지키는 역할을 하고 있습니다.' 오필리어 베이 그란스는 종교 집단의 정신적인 지주로서 그의 능력을 믿고 따르는 신도입니다. 그녀가 자신들을 지키기 위해 수호신의 힘을 빌린다는 이야기를 들은 적이 있습니다. 현재는 교단 내부에서 자신들이 맡고 있는
Response (after) :   '오필리오 베이그란스'()는 황도군에 합류한 후에도 종교 단체에 남아 있었습니다. 그러나 현재는 신도들의 반발로 신도들과 함께 집단 자살을 하는 것으

In [10]:
print("mean:")
display(df_results[["rewards (before)", "rewards (after)"]].mean())
print()
print("median:")
display(df_results[["rewards (before)", "rewards (after)"]].median())

mean:


rewards (before)    2.307434
rewards (after)     3.206085
dtype: float64


median:


rewards (before)    2.567383
rewards (after)     3.184570
dtype: float64

## Train 12.8b Model base with A100 one step

In [ ]:
query_txt = "아라드 대륙에 대해서 설명해줘"
query_encode = make_input(query_txt)
query_tensors = tokenizer.encode(query_txt, return_tensors="pt")


response = '아라드 대륙은 벨 마이어 공국, 반투국, 수쥬국, 흑요정 왕국, 데 로스 제국 등 여러 국가가 존재하는 대륙입니다. 이 대륙은 아라드 대륙이라는 이름으로 알려져 있습니다. 이 대륙에는 다양한 종족들이 살고 있으며, 모험가들은 다양한 종족들과 교류하며 모험을 즐길 수 있습니다. 아라드 대륙에서는 다양한 문화와 역사를 경험할 수 있으며, 다양한 사람들과의 교류를 통해 새로운 경험을 할 수 있을 것입니다. 또한, 이 대륙에서는 수많은 유적과'
response_tensors = tokenizer.encode(response, return_tensors="pt")

rewards = [torch.tensor(3.47)]

ppo_trainer.step([query_tensors[0]], [response_tensors[0]], rewards)


{'objective/kl': 56.351043701171875,
 'objective/kl_dist': 56.351043701171875,
 'objective/logprobs': array([[-1.45925608e+01, -2.46456456e+00, -1.60551977e+00,
         -5.56262493e+00, -3.77504015e+00, -1.73158407e+00,
         -5.53011179e+00, -9.62687874e+00, -2.19370961e-01,
         -3.63981158e-01, -7.25596130e-01, -5.28046417e+00,
         -7.88754225e-02, -4.61143218e-02, -8.07286054e-02,
         -8.75985846e-02, -1.96038413e+00, -3.66744590e+00,
         -1.24913186e-01, -1.84856510e+00, -9.54421908e-02,
         -4.05263805e+00, -2.81907648e-01, -2.47410852e-02,
         -5.21565497e-01, -2.53694940e+00, -5.41553088e-03,
         -4.58302610e-02, -9.91374731e-01, -7.24837780e-01,
         -2.68632722e+00, -2.54865885e-01, -5.36051532e-03,
         -2.25284767e+00, -1.47872102e+00, -1.14727473e+00,
         -7.67057896e-01, -1.82066262e+00, -5.62259138e-01,
         -2.25619271e-01, -5.44962406e-01, -1.39080846e+00,
         -3.23882282e-01, -2.24916553e+00, -1.32182491e+00,